In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\USER\Desktop\my_git\pof-model-test\DL_FIRE_M-C61_613954\fire_nrt_M-C61_613954.csv')
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,36.98479,126.69270,300.88,1.24,1.11,2025-02-01,1227,Terra,MODIS,35,6.1NRT,274.68,12.68,N
1,36.98031,126.68984,301.82,1.35,1.15,2025-02-02,520,Aqua,MODIS,37,6.1NRT,282.99,10.43,D
2,36.98298,126.70437,303.46,1.35,1.15,2025-02-02,520,Aqua,MODIS,53,6.1NRT,283.87,11.85,D
3,37.55538,127.05664,302.60,1.34,1.15,2025-02-04,145,Terra,MODIS,50,6.1NRT,269.74,15.89,D
4,36.98527,126.69640,305.90,1.01,1.00,2025-02-05,540,Aqua,MODIS,35,6.1NRT,278.31,9.84,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,35.36948,129.25702,305.25,1.60,1.25,2025-04-29,134,Terra,MODIS,53,6.1NRT,293.42,12.54,D
1016,35.32056,129.20131,303.33,1.60,1.24,2025-04-29,134,Terra,MODIS,27,6.1NRT,293.08,10.00,D
1017,36.97812,126.70195,323.67,1.02,1.01,2025-04-29,550,Aqua,MODIS,70,6.1NRT,303.86,14.78,D
1018,36.98523,126.69556,314.79,1.00,1.00,2025-04-29,1234,Terra,MODIS,90,6.1NRT,286.93,15.71,N


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    1020 non-null   float64
 1   longitude   1020 non-null   float64
 2   brightness  1020 non-null   float64
 3   scan        1020 non-null   float64
 4   track       1020 non-null   float64
 5   acq_date    1020 non-null   object 
 6   acq_time    1020 non-null   int64  
 7   satellite   1020 non-null   object 
 8   instrument  1020 non-null   object 
 9   confidence  1020 non-null   int64  
 10  version     1020 non-null   object 
 11  bright_t31  1020 non-null   float64
 12  frp         1020 non-null   float64
 13  daynight    1020 non-null   object 
dtypes: float64(7), int64(2), object(5)
memory usage: 111.7+ KB


## 전처리

In [8]:
# confidence 30 이하 모두 제거
df = df[df.confidence >= 30]

# 날짜 형식 변환
df['acq_date'] = pd.to_datetime(df['acq_date'])

# 0.1도 격자 ID 생성
df['lon_bin'] = np.floor(df.longitude / 0.1).astype(int)
df['lat_bin'] = np.floor(df.latitude / 0.1).astype(int)

# 전 지구 고유 ID (위/경 0.1도 셀을 행번호처럼 사용)
# 행 개수 = 1,800개 (위도)
# 열 개수 = 3,600개 (경도)
df['grid_id'] = (df.lat_bin + 900) * 3600 + (df.lon_bin + 1800)

# 하루 및 격자별 열접 존재 여부 -> af_flag
af = (df.groupby(['acq_date', 'grid_id']).size().reset_index(name='cnt').assign(af_flag=1)[['acq_date', 'grid_id', 'af_flag']])

# 음성(0) 샘플 채우기 - 0.1도 격자 목록 & 날짜 달력과 outer-join
# 그리드 목록, 달력은 별도로 생성

grid_df = pd.DataFrame({'grid_id': np.unique(df.grid_id)})
dates_df = pd.DataFrame({'acq_date': pd.date_range("2024-01-01", "2025-04-30")})
full = (dates_df.assign(key=1).merge(grid_df.assign(key=1), on='key').drop('key', axis=1))
target = (full.merge(af, on=['acq_date', 'grid_id'], how='left').fillna({'af_flag':0}).astype({'af_flag':'uint8'}))

# 저장
target.to_csv(
    "af_flag_korea.csv",
    index=False,          # 행 번호 저장 X
    encoding="utf-8"
)

In [12]:
df = pd.read_csv(r"C:\Users\USER\Desktop\my_git\pof-model-test\ml_modeling\af_flag_korea.csv")
print(df["af_flag"].value_counts())
display(df.head())

af_flag
0    74548
1      296
Name: count, dtype: int64


,acq_date,grid_id,af_flag
0,2024-01-01,4488667,0
1,2024-01-01,4495870,0
2,2024-01-01,4495879,0
3,2024-01-01,4499472,0
4,2024-01-01,4499474,0


In [13]:
flag_df = df[df['af_flag']==1]
print(flag_df)

         acq_date  grid_id  af_flag
61258  2025-02-01  4571466        1
61412  2025-02-02  4571466        1
61413  2025-02-02  4571467        1
61743  2025-02-04  4593070        1
61874  2025-02-05  4571466        1
...           ...      ...      ...
74658  2025-04-29  4571468        1
74664  2025-04-29  4575069        1
74669  2025-04-29  4578681        1
74670  2025-04-29  4578684        1
74681  2025-04-29  4603872        1

[296 rows x 3 columns]


In [ ]:
import pandas as pd
import folium

def plot_fire_map(csv_path, target_date):
    # CSV 로드 및 필터
    df = pd.read_csv(csv_path, parse_dates=['acq_date'])
    
    # 해당 날짜의 양성 레코드만
    subset = df[(df['acq_date'] == pd.to_datetime(target_date)) & (df['af_flag'] == 1)].copy()
    
    # grid_id → 위경도 계산
    subset['lat_bin'] = subset['grid_id'] // 3600 - 900
    subset['lon_bin'] = subset['grid_id'] % 3600 - 1800
    subset['latitude'] = (subset['lat_bin'] + 0.5) * 0.1
    subset['longitude'] = (subset['lon_bin'] + 0.5) * 0.1

    # Folium 지도 생성 (한반도 중심)
    m = folium.Map(location=[36, 128], zoom_start=6, tiles='OpenStreetMap')
    
    # 화점 표시
    for _, row in subset.iterrows():
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=6,
            color='red',
            fill=True,
            fill_opacity=0.8
        ).add_to(m)
    
    return m

# 사용 예시: csv_path를 실제 위치로 바꾸고 실행하세요.
m = plot_fire_map(
    'C:/Users/USER/Desktop/my_git/pof-model-test/ml_modeling/af_flag_korea.csv',
    '2025-03-26'
)
m